In [1]:
import cv2
import numpy as np
from os.path import abspath
from os import listdir
import random
from scipy.stats import norm
from tqdm import tqdm

In [2]:
def pixelate(image, blocks=None):
    # divide the input image into NxN blocks
    if blocks is None:
        blocks = np.random.randint(120,151)
    (h, w) = image.shape[:2]
    xSteps = np.linspace(0, w, blocks + 1, dtype="int")
    ySteps = np.linspace(0, h, blocks + 1, dtype="int")
    # loop over the blocks in both the x and y direction
    for i in range(1, len(ySteps)):
        for j in range(1, len(xSteps)):
            # compute the starting and ending (x, y)-coordinates
            # for the current block
            startX = xSteps[j - 1]
            startY = ySteps[i - 1]
            endX = xSteps[j]
            endY = ySteps[i]
            roi = image[startY:endY, startX:endX]
            (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image, (startX, startY), (endX, endY), (B, G, R), -1)
    # return the pixelated blurred image
    return image


def generate_random_lines(imshape, drop_length):
    drops = []
    area = imshape[0] * imshape[1]
    drops_num = area//np.random.randint(600, 700)
    slant = np.random.randint(-10, 10)

    for i in range(drops_num): ## If You want heavy rain, try increasing this
        if slant<0:
            x1 = np.random.randint(slant, imshape[1])
        else:
            x1 = np.random.randint(0, imshape[1]-slant)
        drop_length = np.random.randint(10, 60)
        y1 = np.random.randint(0, imshape[0]-drop_length)
        x2 = x1 + slant
        y2 = y1 + drop_length
        drops.append((x1, y1, x2, y2))
    
    return drops


def rain_process(image, rain_drops, drop_color):
    imshape = image.shape  
    image_t = image.copy()
    for rain_drop in rain_drops:
        pt1 = (rain_drop[0], rain_drop[1])
        pt2 = (rain_drop[2], rain_drop[3])
        drop_width = np.random.randint(1, 3)
        cv2.line(image_t, pt1, pt2, drop_color, drop_width)
    
#     k = np.random.randint(4, 8)
#     image = cv2.blur(image_t, (k, k))
    
    return image_t


def add_rain(image, slant=-1, drop_length=20, drop_color=(200,200,200)):
    imshape = image.shape
    slant_ex = slant
    if slant_ex == -1:
        slant = np.random.randint(-10, 10) # generate random slant if no slant value is given
    rain_drops = generate_random_lines(imshape, drop_length)
    output = rain_process(image, rain_drops, drop_color)

    return output

In [3]:
def generate_spot_light_mask(mask_size,
                             position=None,
                             max_brightness=255,
                             min_brightness=0,
                             mode="gaussian",
                             linear_decay_rate=None,
                             speedup=False,
                             reverse=False):
    """
    Generate decayed light mask generated by spot light given position, direction. Multiple spotlights are accepted.
    Args:
        mask_size: tuple of integers (w, h) defining generated mask size
        position: list of tuple of integers (x, y) defining the center of spotlight light position,
                  which is the reference point during rotating, format=[(w1,h1), (w2,h2)]
        max_brightness: integer that max brightness in the mask
        min_brightness: integer that min brightness in the mask
        mode: the way that brightness decay from max to min: linear or gaussian
        linear_decay_rate: only valid in linear_static mode. Suggested value is within [0.2, 2]
        speedup: use `shrinkage then expansion` strategy to speed up vale calculation
    Return:
        light_mask: ndarray in float type consisting value from max_brightness to min_brightness. If in 'linear' mode
                    minimum value could be smaller than given min_brightness.
    """
    if position is None:
        position = [(random.randint(0, mask_size[0]), random.randint(0, mask_size[1]))]
    if linear_decay_rate is None:
        if mode == "linear_static":
            linear_decay_rate = random.uniform(0.25, 1)
    assert mode in ["linear", "gaussian"], \
        "mode must be linear_dynamic, linear_static or gaussian"
    mask = np.zeros(shape=(mask_size[1], mask_size[0]), dtype=np.float32)
    if mode == "gaussian":
        mu = np.sqrt(mask.shape[0]**2+mask.shape[1]**2)
        dev = mu / 3.5
        mask = _decay_value_radically_norm_in_matrix(mask_size, position, max_brightness, min_brightness, dev)
    mask = np.asarray(mask, dtype=np.uint8)
    # add median blur
    mask = cv2.medianBlur(mask, 5)
    if reverse:
        mask = 255 - mask
    return mask

def _decay_value_radically_norm_in_matrix(mask_size, centers, max_value, min_value, dev):
    """
    _decay_value_radically_norm function in matrix format
    """
    center_prob = norm.pdf(0, 0, dev)
    x_value_rate = np.zeros((mask_size[1], mask_size[0]))
    for center in centers:
        coord_x = np.arange(mask_size[0])
        coord_y = np.arange(mask_size[1])
        xv, yv = np.meshgrid(coord_x, coord_y)
        dist_x = xv - center[0]
        dist_y = yv - center[1]
        dist = np.sqrt(np.power(dist_x, 2) + np.power(dist_y, 2))
        x_value_rate += norm.pdf(dist, 0, dev) / center_prob
    mask = x_value_rate * (max_value - min_value) + min_value
    mask[mask > 255] = 255
    return mask

def add_spot_light(image, light_position=None, max_brightness=255, min_brightness=0,
                   mode='gaussian', linear_decay_rate=None, transparency=None, reverse=False):
    """
    Add mask generated from spot light to given image
    """
    if transparency is None:
        transparency = random.uniform(0.5, 0.85)
    frame = image
    height, width, _ = frame.shape
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = generate_spot_light_mask(mask_size=(width, height),
                                    position=light_position,
                                    max_brightness=max_brightness,
                                    min_brightness=min_brightness,
                                    mode=mode,
                                    linear_decay_rate=linear_decay_rate,
                                    reverse=reverse)
    hsv[:, :, 2] = hsv[:, :, 2] * transparency + mask * (1 - transparency)
    frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    frame[frame > 255] = 255
    frame = np.asarray(frame, dtype=np.uint8)
    return frame


def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    if gamma == 1.0:
        gamma = np.random.randint(40,91) / 100.0
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [12]:
load_path1 = './result2/img/'
load_path2 = './result3/img/'
save_path1 = './result2/img_blur/'
save_path2 = './result3/img_blur/'

load = [load_path1, load_path2]
save = [save_path1, save_path2]

for l, s in zip(load, save):
    files = listdir(l)
    loader = tqdm(files, bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', desc=l)
    for f in loader:
        if f.endswith('jpg'):
            img = cv2.imread(l+f)
            img = add_rain(img)
            img = add_spot_light(img, reverse=True)
            img = adjust_gamma(img)
            img = add_spot_light(img, max_brightness=220)
            img = pixelate(img)
            cv2.imwrite(s+f, img)
    loader.close()

  0%|                    | 0/10000 [00:00<?, ?it/s]                                                                                                                                                                           

Processing:  ./result2/img/


100%|████████████████████| 10000/10000 [45:02<00:00,  3.70it/s]                                                                                                                                                               


Processing:  ./result3/img/


100%|████████████████████| 20000/20000 [1:19:01<00:00,  4.22it/s]                                                                                                                                                             


In [4]:
path = './unit/2_img/'
files = listdir(path)
pbar = tqdm(files, ncols=80, desc='Proccessing')
for f in pbar:
    if f.endswith('jpg'):
        flag1 = (random.randint(1, 10)>5)
        flag2 = (random.randint(1, 10)>5)
        if flag1 or flag2:
            img_path = path + f
            img = cv2.imread(img_path)
            if flag1:
                img = add_spot_light(img, reverse=True)
            if flag2:
                img = adjust_gamma(img)
                img = add_spot_light(img, max_brightness=220)
            cv2.imwrite(img_path, img)
        else:
            pass

Proccessing: 100%|██████████████████████| 240000/240000 [42:11<00:00, 94.79it/s]
